In [ ]:
# install necessary python libraries 
%pip install python-bitcoinrpc
%pip install paramiko

In [48]:
# read credentials from file 
import pandas as pd

credentials = pd.read_json('credentials.json')

In [49]:
# establish ssh connection to the host 
import paramiko 
host = "artemis.uni.ma"
port = 22
username = credentials['user'][0]
password = credentials['pwd'][0]

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

In [ ]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getbestblockhash") # returns the header hash of the most recent block on the best block chain.
print(stdout.read().decode("utf-8")) 

# Command: bitcoin-cli getblock [blockhash] 1
Get blocks via the command `bitcoin-cli getblock [blockhash] 1`. The output contains information such as the transactions belonging to this block as well as the `previousblockhash` and `nextblockhash`. Via these attributes blocks can be retrieved in an ordered fashion and we have the neccessary information about the `PRECEDES/BELONGS_TO` relationship. 
In addition, this command returns the `blockheight` and `blocktime` (e.g., 1619858709) and an array of `txids` which are associated/ stored with the block. 

In [ ]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblock 00000000000000000008c69128a76fb9888164db31dace993cf2a6b8a574664a 1")
print(stdout.read().decode("utf-8")) 

# Command: bitcoin-cli getrawtransaction [txid] true
Number of objects contained in `vin` equals `inDegree`(?)
Number of objects contained in `vout` equals `outDegree`(?) 
returns info about which block in belongs => `BELONG_TO` relationship


In [ ]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getrawtransaction 3511d57b5d64df0c1ba7c8b9d16ecdad7ea032e31003ac76abb9118e50ccea67 true") 

# parse json output
import json
rawtx = json.loads(stdout.read().decode("utf-8"))

print("Indegree: ", len(rawtx['vin']))
print("Outdegree: ", len(rawtx['vout']))

inSum = 0 
for vin in rawtx['vin']:
    print(vin['txid'])
    inSum += vin['vout']
    
print("InSum: ", inSum)
    
outSum = 0
for v in rawtx['vout']:
    outSum += v['value']
    for a in v['scriptPubKey']['addresses']:
        print(a)
    
    
print("OutSum: ", outSum)

In [ ]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getrawtransaction 3511d57b5d64df0c1ba7c8b9d16ecdad7ea032e31003ac76abb9118e50ccea67 true")
print(stdout.read().decode("utf-8")) 


In [ ]:
# address - tx.outputs[o].addresses[0].address
# blocks - [block_hash, block_height, block_timestamp]
# transaction - txid, block_date, inDegree, outDegree

In [ ]:
# before relationship - ([previous_block_hash, block_hash, 'PRECEDES']) OK
# belongs relationship - txid, block_hash, 'BELONGS_TO' OK
# receives relationship - [tx_id, val, o, addr, 'RECEIVES'] o= index array OK
# sends relationship - ([in_address, in_value, tx_id, 'SENDS'])

# Complete Process
continuously monitor the bestblockhash - returns the tip of the blockchain

In [ ]:
# get best blockhash - returns the header hash of the most recent block on the best block chain
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getbestblockhash")
bestblockhash = stdout.read().decode("utf-8")
print(bestblockhash) 

In [ ]:
# get blockinfo 
command = "bitcoin-cli getblock " + bestblockhash + " 2"
#stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblock 00000000000000000008bb27dd339ad1b2534f9060907a2be6925bb375395ba2 1")
stdin, stdout, stderr = ssh.exec_command(command)
block = json.loads(stdout.read().decode("utf-8"))

# before relationship - ([previous_block_hash, block_hash, 'PRECEDES']) OK
previousblockhash = block['previousblockhash']
nextblockhash = block['nextblockhash']

print("previousblockhash: ", previousblockhash)
print("nextblockhash: ", nextblockhash)

# blocks - [block_hash, block_height, block_timestamp]
from datetime import datetime, timezone
ts_epoch = block['time']
block_timestamp = datetime.fromtimestamp(ts_epoch)
block_date = block_timestamp.strftime('%Y-%m-%d')
block_hash = block['hash']
block_height = block['height']

print("block_hash: ", block_hash)
print("block_height: ", block_height)
print("block_timestamp: ", block_timestamp)
print("block_date: ", block_date) 





# Transaction Input Address
The data you are looking for isn't actually included in the transaction that spends the UTXOs. Addresses are basically shorthand for a scriptPubKey, and scriptPubKeys are associated with outputs, not inputs. So if you want to check if an input spends from a certain address, you have to get the corresponding previous output and look what address it pays to.

In [ ]:
# parse transactions which are contained in one block 

# belongs relationship - txid, block_hash, 'BELONGS_TO' OK => all these transactions belong to the same block
# transaction - txid, block_date, inDegree, outDegree

for tx in block['tx']: 
    print(tx, block_hash, 'BELONGS_TO')
    print("\n-----TxID: ", tx)
    command = "bitcoin-cli getrawtransaction " + tx + " true"
    stdin, stdout, stderr = ssh.exec_command(command)
    rawtx = json.loads(stdout.read().decode("utf-8"))
    print("Indegree: ", len(rawtx['vin']))
    print("Outdegree: ", len(rawtx['vout']))
    
    # ([in_address, in_value, tx_id, 'SENDS'])
    for vin in rawtx['vin']:
        try: 
            print("TxID Input: ", vin['txid'])
        except Exception as e:
            pass
        break;
    
    # receives relationship - [tx_id, val, o, addr, 'RECEIVES'] o= index array OK
    for v in rawtx['vout']:
        o = v['n']
        val = v['value']
        print("\n#Addresses: ", len(v['scriptPubKey']['addresses']))
        if len(v['scriptPubKey']['addresses'])> 0: 
            for a in v['scriptPubKey']['addresses']:
                print(a)
        break;



## Get Input Addresses

In [57]:
# parse transaction data 
import json
txid = "00000005aca88ceece655e19070dbfe9416b0c2850da0463f1e4c823bb41f295"
txid = "00000990d69e3693c9ccf3f90b20f716ddcd1d043c7c613a9d58f5a722d06648"
txid = "0000000fdb4d9b9479d9b061921cb9d073f1c814a3bbef30ccae212ab651952d"
command = "bitcoin-cli getrawtransaction " + txid + " true"
stdin, stdout, stderr = ssh.exec_command(command)
rawtx = json.loads(stdout.read().decode("utf-8"))

In [66]:
address_list = []  

outSum = 0

for o in rawtx['vout']: 
    outSum += o['value']
    # check whether there are output adresses
    if 'addresses' in o['scriptPubKey']:
        for a in o['scriptPubKey']['addresses']: 
            # a Python object (dict):
            addrObj = {
               "addr": a,
                "output_nr": o['n']
            }
            jAddr = json.dumps(addrObj)
            # convert json string to Json Dict
            jsonDict = json.loads(jAddr)
            address_list.append(jsonDict)

# print json object = one transaction within a block
txdata = {}
txdata['txid'] = txid
#txdata['block_hash'] = block_hash
#txdata['block_date'] = str(block_date)
txdata['address_list'] = address_list
txdata['inDegree'] = len(rawtx['vin']) 
txdata['outDegree'] = len(rawtx['vout'])
x = round(outSum*100000000, 0)
txdata['outSum'] = int( x)



# creating input list  ## Throwing error for 
input_list = []
for i in rawtx['vin']: 
    # bypassing coinbase transactions!
    try:
        addrObj = {
           "txid": i['txid'],
            "output_nr": i['vout']
        }
        jAddr = json.dumps(addrObj)
        jsonDict = json.loads(jAddr)
        input_list.append(jsonDict)

        txdata['input_list'] = input_list

    # reformulate and handle coinbase!
    except: pass

    tx_json_data = json.dumps(txdata, indent=4, sort_keys=False)
    

print("#######Transaction Output#########")
print(tx_json_data)    
    

#######Transaction Output#########
{
    "txid": "0000000fdb4d9b9479d9b061921cb9d073f1c814a3bbef30ccae212ab651952d",
    "address_list": [
        {
            "addr": "1Nvv2vdKssYTJNPYkqmsHHf9Kn7ei4nbXu",
            "output_nr": 0
        },
        {
            "addr": "1gucZbN2LdKr2z3szVWx1ZiQNeYgXK6py",
            "output_nr": 1
        }
    ],
    "inDegree": 2,
    "outDegree": 2,
    "outSum": 2431925,
    "input_list": [
        {
            "txid": "11f3cdf4ed881811d5cf710e7fb8e6115768c999990a35b8f092e5e94adfbd5d",
            "output_nr": 0
        },
        {
            "txid": "e365bf7ee8f4466096743d53b327cddc2054df284f46907abc9a84477f91470d",
            "output_nr": 1
        }
    ]
}


In [72]:
# get input addresses
outSum = 0
input_address_list = []
for i in input_list:
    #print(i)
    command = "bitcoin-cli getrawtransaction " + i['txid'] + " true"
    stdin, stdout, stderr = ssh.exec_command(command)
    inputtx = json.loads(stdout.read().decode("utf-8"))
    inputtx_json_data = json.dumps(inputtx, indent=4, sort_keys=False)
    
    val = int (inputtx['vout'][i['output_nr']]['value']*100000000)
    outSum += val 
    
    #input_address_list = []
    #print(len(inputtx['vout'][i['output_nr']]['scriptPubKey']['addresses']))
    for ia in inputtx['vout'][i['output_nr']]['scriptPubKey']['addresses']: 
        input_address_list.append(ia)
       

print("Input Address List: ", input_address_list)

print("Insum: ", outSum)
  

Input Address List:  ['19QgnpbWqopBV5Y53C7AU5bgkpXkLofybH', '1FzBd1p9uKXZcwqTHhAQzS3XCeUuK8jxHR']
Insum:  2439567
